# Writing a LogPDF

Probability density functions in Pints can be defined via [models](writing-a-model.ipynb) and [problems](inference-first-example.ipynb), but they can also be defined directly.

In this example, we implement the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function) and run an [optimisation](optimisation-first-example.ipynb) using it.

The rosenbrock function is a two dimensional defined as

```
f(x,y) = -((a - x)^2 + b(y - x^2)^2)
```

where ``a`` and ``b`` are constants and ``x`` and ``y`` are variable. In analogy with typical Pints models ``x`` and ``y`` are our _parameters_.

First, take a look at the [LogPDF](http://pints.readthedocs.io/en/latest/log_pdfs.html#pints.LogPDF) interface. It tells us two things:

1. We need to add a `dimension()` method that tells pints the dimension of the problem space
2. Objects of our class should be _callable_. In Python, we can do this using the [special method](https://docs.python.org/3/reference/datamodel.html#object.__call__) `__call__`.

The input to this method should be a vector, so we should rewrite it as

```
f(p) = -((a - p[0])^2 + b(p[1] - p[0]^2)^2)
```



The result of calling this method should be the logarithm of a normalised log likelihood. That means we should (1) take the logarithm of ``f`` instead of returning it directly, and (2) invert the method, so that it has a clearly defined _maximum_ that we can search for.

So we should create an object that evaluates

```
-log(f(p))
```

We now have all we need to implement a ```Rosenbrock``` class:

In [1]:
import numpy as np
import pints

class Rosenbrock(pints.LogPDF):
    def __init__(self, a=1, b=100):
        self._a = a
        self._b = b

    def __call__(self, x):
        return - np.log((self._a - x[0])**2 + self._b * (x[1] - x[0]**2)**2)

    def dimension(self):
        return 2

We can test our class by creating an object and calling it with a few parameters:

In [2]:
r = Rosenbrock()
print(r([0, 0]))
print(r([0.1, 0.1]))
print(r([0.4, 0.2]))

-0.0
-0.482426149244
0.653926467407


Wikipedia tells for that for ``a = 1`` and ``b = 100`` the minimum value should be at ``[1, 1]``. We can test this by inspecting its value at that point:

In [3]:
r([1, 1])

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


inf

We get an error here, because the notebook doesn't like it, but it returns the correct value!

Now let's try an optimisation:

In [4]:
# Define some boundaries
boundaries = pints.Boundaries([-5, -5], [5, 5])

# Pick an initial point
x0 = [2, 2]

# And run!
xbest, fbest = pints.optimise(r, x0, boundaries=boundaries)

Maximising LogPDF
using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 6
0: -3.00283570329
1: -2.95379513606
2: 3.00251004185
3: 3.00251004185
20: 3.00251004185
40: 3.00251004185
60: 5.45979020078
80: 13.3997123147
100: 26.7688429516
120: 33.690992939
140: 39.3055571977
160: 47.3209906429
180: 56.4466153132
200: 67.0358494896
220: 73.4736011394
240: inf
260: inf
280: inf
300: inf
320: inf


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


340: inf
360: inf
380: inf
400: inf
420: inf
Halting: No significant change for 200 iterations.
427: inf


Finally, print the returned point. If it worked, we should be at `[1, 1]`:

In [5]:
print(xbest)

[ 1.  1.]
